In [3]:
import torch
import numpy as np
import glob
from bs4 import BeautifulSoup
from flair.data import Sentence
from segtok.segmenter import split_single
import random
from flair.models import SequenceTagger
import xml.etree.ElementTree as ET
import string
import re
import json
import random
from pycorenlp import StanfordCoreNLP


In [4]:
nlp = StanfordCoreNLP('http://localhost:9000')
properties={
  'annotators': 'ssplit',
  'outputFormat': 'json'
  }
def sentence_split(text, properties={'annotators': 'ssplit', 'outputFormat': 'json'}):
    """Split sentence using Stanford NLP"""
    annotated = nlp.annotate(text, properties)
    sentence_split = list()
    for sentence in annotated['sentences']:
        s = [t['word'] for t in sentence['tokens']]
        k = [item.lower() for item in s if item not in [",", ".", '...', '..']]
        sentence_split.append(" ".join(k))
    return sentence_split

['hello all', 'my name is titipat the best lol player']

In [26]:
class DataLoader :
    def __init__(self, _paths, _hint):
        self.paths = _paths
        self.files = []
        self.inputs = []
        self.data = []
        self.train = []
        self.valid = []
        self.test = []
        self.hintFile = _hint
        self.splitTest = []
        self.splitValid = []
        self.splitTrain = []
        self.postags = []
        self.tagger = SequenceTagger.load('pos')

    def listFiles(self):
        for path in self.paths:
            self.files = self.files + glob.glob(path + "*.sgm")
        print('Loaded {} files'.format(len(self.files)), end='\n')

    def splitHint(self):
        train = open(self.hintFile + "/train.txt", "r+")
        valid = open(self.hintFile + "/valid.txt", "r+")
        test = open(self.hintFile + "/test.txt", "r+")

        text_train = train.read()
        text_valid = valid.read()
        text_test = test.read()

        list_train = text_train.split("\n")
        for line in list_train:
            self.splitTrain.append(line + ".sgm")

        list_valid = text_valid.split("\n")
        for line in list_valid:
            self.splitValid.append(line + ".sgm")

        list_test = text_test.split("\n")
        for line in list_test:
            self.splitTest.append(line + ".sgm")

    def readData(self):
        print('Starting to Prepare {} files'.format(len(self.files)), end='\n')
        print(" ")
        iteration = 0
        for item in self.files:
            file1 = open(item, "r+")
            text = file1.read()
            sample = {"text": text, "id": item, "parse": BeautifulSoup(text, 'lxml')}
            sample['file_name'] = sample['id'].split("/")[-1]
            if sample['file_name'] in self.splitTrain:
                sample['type'] = "train"
            elif sample['file_name'] in self.splitValid:
                sample['type'] = "valid"
            elif sample['file_name'] in self.splitTest:
                sample['type'] = "test"
            else:
                sample['type'] = "ignore"
            text = sample['parse'].select_one("TEXT").prettify()
            text = re.sub('<[^<]+>', "\n",text)
            text = text.replace("Dr.", "Dr ").replace("Mr.", "Mr ").replace("Al-", "Al").replace("/", " ").replace("'s", " s").replace(".-", "-").replace("-", " ").replace("(", " ").replace(")", " ").replace("%", ' percent ')
            x = re.findall(r'[a-z]\.[A-Z]', text)
            for item in x:
                item = item.replace(".", "\.")
                y = item.replace("\.", " . ")
                text = re.sub(item, y, text)
#             x = re.findall(r'[a-z]-[a-zA-Z]', text)
#             for item in x:
#                 y = item.replace("-", " ")
#                 text = re.sub(item, y, text)
            text = re.sub(' +', ' ', text)
            sample['parse'] = text
            splitted = sentence_split(text)
#             splitted = text.split(".")
#             splitted = [re.sub(' +', ' ', re.sub(r'[^\w\s]',' ',data)).replace(" t ", "'t ") for data in splitted if len(data) > 2]
            splitted = [re.sub(' +', ' ', data).replace("'s", "' s").replace("' t ", "'t ") for data in splitted if len(data) >= 1]
            sentences = [Sentence(re.sub(r'[^\w\s]',' ',sent)) for sent in splitted if sent]
            sentences = [sentence for sentence in sentences if len(sentence) >= 1]
            sample['sentences'] = sentences
            self.inputs.append(sample)
            file1.close()
            iteration += 1
            print('Prepared {} files'.format(iteration), end='\r')
        print('Preparation resulted in {} inputs'.format(len(self.inputs)))

    def prepareLables(self):
        print("starting to prepare lables", end='\n')
        print(" ")
        iteration = 0
        for sample in self.inputs:
            id = sample['id'].split('.sgm')[0]
            path = id + ".apf.xml"
            sample['annotation_path'] = path
            iteration += 1
            print('Prepared {} Lables'.format(iteration), end='\r')

    def Sort(self, sub_li): 
  
        # reverse = None (Sorts in Ascending order) 
        # key is set to sort using second element of  
        # sublist lambda has been used 
        sub_li.sort(key = lambda x: x[3]) 
        return sub_li 
    
    def readLables(self):
        print("starting to read lables", end='\n')
        iteration = 0
        total_ann = 0
        for sample in self.inputs:
            tree = ET.parse(sample['annotation_path']) 
            root = tree.getroot() 
            entities = root.findall(".//entity")
            _all = []
            for entity in entities:
                mentions = entity.findall(".//entity_mention")
                _all.extend([[mention.find("./head/charseq").text, mention.find("./head/charseq").attrib['START'], entity.attrib, mention.attrib['ID'], mention.find("./head/charseq").attrib['END'], mention.find("./extent/charseq").text] for mention in mentions])
            _all = sorted(_all, key=lambda x:int(x[1]))
            entities = [[x[0],x[2]['TYPE'],x[2]['SUBTYPE'],int(x[1]),x[3], int(x[4]), x[5]] for x in _all]
            sample['annotated'] = self.Sort(entities)
            relations = root.findall(".//relation")
            _all = []
            for relation in relations:
                mentions = relation.findall(".//relation_mention")
                _all.extend([[mention.findall("./relation_mention_argument")[0].attrib['REFID'], mention.findall("./relation_mention_argument")[1].attrib['REFID'], relation.attrib, mention.attrib["ID"]] for mention in mentions])
            sample['relations'] = [(x[0],
                                    x[1],
                                    x[2]['TYPE'],
                                    x[2]['SUBTYPE'],
                                    x[3]
                                   )
                                       for x in _all if x[2]['TYPE'] != "METONYMY"]
            sample['relations'].extend([(x[0],
                                        x[1],
                                        x[2]['TYPE'],
                                        "UNKNOWN",
                                        x[3]
                                       )
                                       for x in _all if x[2]['TYPE'] == "METONYMY"])
#             sample['relations'] = _all
            iteration += 1
            total_ann += len(sample['annotated'])
            print('Read {} inputs with total {} annotations'.format(iteration, total_ann), end='\r')

    def match(self, s1, s2):
        if s1 == s2:
            return True
        elif s1 in s2.split("-"):
            return True
        else:
            if s1 == "u.s":
                s1 = "u.s."
            if s1 == "u.k":
                s1 = "u.k."
            if s1 == "u.n":
                s1 = "u.n."
            if s1 == "dr":
                s1 = "dr."
            if s1 == "mr":
                s1 = "mr."
            if s1 == "l":
                s1 == "led"
            if s1 == s2:
                return True
            elif s1 in s2.split("-"):
                return True
            if s2.count(s1):
                if len(s2) >= 3 and (len(s2) <= len(s1) + 1 or len(s1) / len(s2) >= 0.65):
                    return True
                if len(s1) >= 5 and len(s2) >= 8:
                    return True
                if len(s1) <= 4 and s1 == s2[0:len(s1)] or s1 == s2[len(s2)-len(s1): ]:
                    return True
#                 if s1 == "asia" and s2 == "asiaagent":
#                     return True
                    
            s1 = re.sub(r'[^\w\s]','',s1)
            s2 = re.sub(r'[^\w\s]','',s2)
            if s1 == s2:
                return True

        return False
        
    def lableInputs(self):
        print("starting to lable inputs", end='\n')
        print(" ")
        iteration = 0
        self.total_passed = 0
        self.not_checked = 0
        self.not_checked_data = []
        for sample in self.inputs:
            if sample['type'] == "ignore":
                continue
            total_phrases = 0
            passed = 0
            sample['inputs'] = []
            sample['phrases'] = []
            # words array cotains arrays of all sentences inside a file
            checked = 0
            for sentence in sample['sentences']:
                annotated_sentence = []
                check = False
                #                 print(sentence)
                sentence1 = sentence
                sentence = sentence.to_tokenized_string().split(" ")
                phrases = []
                start = 0
                for iteration in range(len(sample['annotated'])):
                    if iteration < checked:
                        continue
                    annotation = sample['annotated'][iteration]
                    if iteration >= 1:
                        if annotation[3] <= sample['annotated'][iteration-1][5]:
                            passed += 1
                            if sample['type'] == "test":
                                self.total_passed += 1
                            checked += 1
                            continue
#                             start = phrases[-1]['before']
                    _list = []
                    text = annotation[0]
                    x = re.findall(r'[a-z]\.[A-Z]', text)
                    for item in x:
                        item = item.replace(".", "\.")
                        y = item.replace("\.", " . ")
                        text = re.sub(item, y, text)
#                     x = re.findall(r'[a-z]-[A-Z]', text)
#                     for item in x:
#                         y = item.replace("-", " ")
#                         text = re.sub(item, y, text)
                    text = text.replace("Al-", "Al").replace("Dr.", "Dr ").replace("Mr.", "Mr ").replace("/", " ").replace("'s", " s").replace(".-", "-").replace("-", " ").replace("%", ' percent ')
                    text = re.sub(' +', ' ', text)
                    try:
                        labels = sentence_split(re.sub(r'[^\w\s]',' ',text))[0]
                    except:
                        print("file name is ", sample['annotation_path'])
                        print("text is ", text)
                        print("iteration is ", iteration)
                        raise
                    labels = Sentence(labels).to_tokenized_string().replace("\n", " ").split(" ")
#                     if labels[0] == "u.s":
#                         labels[0] = "u.s."
#                     if labels[0] == "u.k":
#                         labels[0] = "u.k."
#                     if labels[0] == "u.n":
#                         labels[0] = "u.n."
                    exist_check = False
                    for _itt in range(start, len(sentence)):
                        if self.match(labels[0], sentence[_itt]):
                            exist_check = True
                            value_index = _itt - start
                            if len(labels) > len(sentence[_itt:]):
                                exist_check = False
                            else:
                                for _it in range(len(labels)):
                                    if self.match(labels[_it], sentence[start+value_index+_it]):
                                        _list.append(sentence[start+value_index+_it])
                                    else:
                                        break
                                if len(_list) != len(labels):
                                    _list = []
                                    exist_check = False
                        if exist_check:
                            break
                    if not exist_check:
                        break
                        
                    check = True
                    example = {"text": annotation[0], "label": annotation[1], 'list': _list, 'value_index': value_index, 'before': start, 'id': annotation[4], 'start': annotation[3], 'end' : annotation[5]}

                    for _it in range(start, start+value_index):
                        _input = {"word": sentence[_it], 'w-type': '-O-', 'type': '-O-', 'subtype': '-O-',
                                  'set': sample['type'], 'index': self.index_finder("-O-")}
                        annotated_sentence.append(_input)
                    if len(_list) == 1:
                        _input = {"word": _list[0], 'w-type': annotation[1], 'type': "L-" + annotation[1], 'subtype': annotation[2],
                                          'set': sample['type'], 'start': annotation[3], 'index': self.index_finder(annotation[1])}
                        annotated_sentence.append(_input)
                    elif len(_list) == 2:
                        _input = {"word": _list[0], 'w-type': annotation[1], 'type': "B-" + annotation[1], 'subtype': annotation[2],
                                  'set': sample['type'], 'start': annotation[3], 'index': self.index_finder(annotation[1])}
                        annotated_sentence.append(_input)
                        _input = {"word": _list[1], 'w-type': annotation[1], 'type': "L-" + annotation[1], 'subtype': annotation[2],
                                  'set': sample['type'], 'start': annotation[3], 'index': self.index_finder(annotation[1])}
                        annotated_sentence.append(_input)
                    else:
                        _input = {"word": _list[0], 'w-type': annotation[1], 'type': "B-" + annotation[1], 'subtype': annotation[2],
                                  'set': sample['type'], 'start': annotation[3], 'index': self.index_finder(annotation[1])}
                        annotated_sentence.append(_input)
                        _input = {"word": _list[-1], 'w-type': annotation[1], 'type': "L-" + annotation[1], 'subtype': annotation[2],
                                  'set': sample['type'], 'start': annotation[3], 'index': self.index_finder(annotation[1])}
                        annotated_sentence.append(_input)
                        for item in range(1, len(_list) - 1):
                            _input = {"word": _list[item], 'w-type': annotation[1], 'type': "I-" + annotation[1], 'subtype': annotation[2],
                                      'set': sample['type'], 'start': annotation[3], 'index': self.index_finder(annotation[1])}
                            annotated_sentence.append(_input)
                    start = start + value_index + len(_list)
                    example['after'] = start
                    example['boundary'] = [example['before']+example['value_index'], example['after'] - 1]
                    checked = iteration+1
                    phrases.append(example)
                for _it in range(start, len(sentence)):
                        _input = {"word": sentence[_it], 'w-type': '-O-', 'type': '-O-', 'subtype': '-O-',
                                  'set': sample['type'], 'index': self.index_finder('-O-')}
                        annotated_sentence.append(_input)
                # annotated_sentence is an array of annotated words inside a sentence
                relations = []
                all_relations = []
                phrases_ids = [phrase['id'] for phrase in phrases]
                for phrase in phrases:
                    for rel in sample['relations']:
                        if rel in relations:
                            continue
                        if rel[0] == phrase['id'] and rel[1] in phrases_ids:
                            relations.append(rel)
                        elif (rel[1] not in phrases_ids or rel[0] not in phrases_ids) and (rel[0] == phrase['id'] or rel[1] == phrase['id']):
                            all_relations.append(rel)
#                 if(len(all_relations) != len(relations)):
#                     print("relation not match ", sample['annotation_path'])
                        
                sample['inputs'].append(annotated_sentence)
                sample['phrases'].append(phrases)
                total_phrases += len(phrases)
                if check:
                    self.data.append({"words": annotated_sentence, "sentence": sentence1.to_tokenized_string(), "phrases": phrases, 'relations': relations, 'all_relations': all_relations, 'file': sample['annotation_path']})
                elif sample['type'] == "test":
                    self.not_checked_data.append({"words": annotated_sentence, "sentence": sentence1.to_tokenized_string(), "phrases": phrases, 'relations': relations, 'all_relations': all_relations, 'file': sample['annotation_path']})
                    self.not_checked += 1
                    self.data.append({"words": annotated_sentence, "sentence": sentence1.to_tokenized_string(), "phrases": phrases, 'relations': relations, 'all_relations': all_relations, 'file': sample['annotation_path']})
            iteration += 1
            if(total_phrases + passed != len(sample['annotated'])):
                print("phrase not match ", sample['annotation_path'])
            print('Labled {} inputs with total number of sentences with entities of {} sentences'.format(iteration, len(
                self.data)), end='\r')
        self.data = [item for item in self.data if len(item["words"]) != 0]
        
    def index_finder(self, _class, choices=["ORG", "FAC", "PER", "VEH", "LOC", "WEA", "GPE", "-O-"]):
        return choices.index(_class)

    def posTagFinder(self):
        _it = 0
        for _it in range(1500):
            item = random.choice(self.data)
            temp = Sentence(item['sentence'])
            self.tagger.predict(temp)
            _dict = temp.to_dict(tag_type='pos')
            self.postags.extend([sample['type'] for sample in _dict['entities'] if sample['type'] not in self.postags])
        self.postags = list(set(self.postags))
    
    def fire(self):
        self.listFiles()
        self.splitHint()
        self.readData()
        self.prepareLables()
        self.readLables()
        self.lableInputs()
#         self.posTagFinder()


In [19]:
data_path = [
     "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/bc/fp1/",
     "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/bc/fp2/",
     "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/bn/fp1/",
     "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/bn/fp2/",
#      "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/cts/fp1/",
#      "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/cts/fp2/",
     "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/nw/fp1/",
     "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/nw/fp2/",
#      "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/un/fp1/",
#      "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/un/fp2/",
     "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/wi/fp1/",
     "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/wi/fp2/", 
]

hint_path = "/home/hfaghihi/LDC2006T06/split"

data = DataLoader(data_path, hint_path)
data.fire()

2020-01-09 19:40:12,056 loading file /home/hfaghihi/.flair/models/en-pos-ontonotes-v0.4.pt
Loaded 860 files
Starting to Prepare 860 files
 
Preparation resulted in 860 inputs
starting to prepare lables
 
starting to read lables
starting to lable inputsal 68609 annotations
 


In [28]:
print(data.total_passed)
print(data.not_checked)
print(data.not_checked_data[1])
number = 0
for item in data.data:
    if not len(item['phrases']):
        number += 1
print(number)

6
281
{'words': [{'word': 'they', 'w-type': '-O-', 'type': '-O-', 'subtype': '-O-', 'set': 'test', 'index': 7}, {'word': 'come', 'w-type': '-O-', 'type': '-O-', 'subtype': '-O-', 'set': 'test', 'index': 7}, {'word': 'in', 'w-type': '-O-', 'type': '-O-', 'subtype': '-O-', 'set': 'test', 'index': 7}, {'word': 'a', 'w-type': '-O-', 'type': '-O-', 'subtype': '-O-', 'set': 'test', 'index': 7}, {'word': 'bag', 'w-type': '-O-', 'type': '-O-', 'subtype': '-O-', 'set': 'test', 'index': 7}], 'sentence': 'they come in a bag', 'phrases': [], 'relations': [], 'all_relations': [], 'file': '/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/bc/fp1/CNN_IP_20030402.1600.02-2.apf.xml'}
281


In [ ]:
for item in data.inputs:
    if item['annotation_path'] == "/home/hfaghihi/LDC2006T06/ace_2005_td_v7/data/English/bc/fp1/CNN_IP_20030402.1600.02-2.apf.xml":
        sample = item
        break

In [ ]:
sample['inputs'] = []
sample['phrases'] = []
total_phrases = 0
# words array cotains arrays of all sentences inside a file
checked = 0
passed = 0
passed_items = []
before_passed = []
for sentence in sample['sentences']:
    annotated_sentence = []
    check = False
    #                 print(sentence)
    sentence1 = sentence
    sentence = sentence.to_tokenized_string().split(" ")
    print("sentence is ", sentence)
    phrases = []
    start = 0
    count = 0
    for iteration in range(len(sample['annotated'])):
        count += 1
        if iteration < checked:
            continue
        annotation = sample['annotated'][iteration]
        if iteration >= 1:
            if annotation[3] <= sample['annotated'][iteration-1][5]:
                passed += 1
                checked += 1
                before_passed.append(sample['annotated'][iteration-1])
                passed_items.append(annotation)
                continue
        _list = []
        text = annotation[0]
        x = re.findall(r'[a-z]\.[A-Z]', text)
        for item in x:
            item = item.replace(".", "\.")
            y = item.replace("\.", " . ")
            text = re.sub(item, y, text)
        x = re.findall(r'[a-z]-[A-Z]', text)
        for item in x:
            y = item.replace("-", " ")
            text = re.sub(item, y, text)
        text = text.replace("Al-", "Al").replace("Dr.", "Dr ").replace("Mr.", "Mr ").replace("/", " ").replace("'s", " s").replace(".-", "-").replace("-", " ").replace("%", ' percent ')
        text = re.sub(' +', ' ', text)
        labels = sentence_split(re.sub(r'[^\w\s]',' ',text))[0]
        print("labels are ", labels)
        labels = Sentence(labels).to_tokenized_string().replace("\n", " ").split(" ")
        print(labels[0])

#         if labels[0] == "u.s":
#             labels[0] = "u.s."
#         if labels[0] == "u.k":
#             labels[0] = "u.k."
#         if labels[0] == "u.n":
#             labels[0] = "u.n."
        exist_check = False
        for _itt in range(start, len(sentence)):
            checking = False
            if data.match(labels[0], sentence[_itt]):
                exist_check = True
#                             try:
                value_index = _itt - start
#                             except:
#                                 print("error")
#                                 print(labels[0])
#                                 print(sample['annotation_path'])
#                                 print(sentence[start:])
#                                 raise
                if len(labels) > len(sentence[_itt:]):
                    print("here surprise")
                    exist_check = False
                else:
                    for _it in range(len(labels)):
                        if data.match(labels[_it], sentence[start+value_index+_it]):
                            _list.append(sentence[start+value_index+_it])
                        else:
                            print("not matched ", labels[_it], " and ", sentence[start+value_index+_it])
                            break
                    if len(_list) != len(labels):
                        print(_list)
                        print(labels)
                        _list = []
                        print("here 4")
                        exist_check = False
            if exist_check:
                break
        if not exist_check:
            print("here 3")
            break

#                     if labels[0] in sentence[start:]:
#                         value_index = sentence[start:].index(labels[0])
#                         if len(labels) > len(sentence[value_index:]):
#                             break
#                     else:
#                         break

        
        check = True
#         print("here 6")
        example = {"text": annotation[0], "label": annotation[1], 'list': _list, 'value_index': value_index, 'before': start, 'id': annotation[4], 'start': annotation[3]}

        for _it in range(start, start+value_index):
            _input = {"word": sentence[_it], 'w-type': '-O-', 'type': '-O-', 'subtype': '-O-',
                      'set': sample['type'], 'index': data.index_finder("-O-")}
            annotated_sentence.append(_input)
        if len(_list) == 1:
            _input = {"word": _list[0], 'w-type': annotation[1], 'type': "L-" + annotation[1], 'subtype': annotation[2],
                              'set': sample['type'], 'start': annotation[3], 'index': data.index_finder(annotation[1])}
            annotated_sentence.append(_input)
        elif len(_list) == 2:
            _input = {"word": _list[0], 'w-type': annotation[1], 'type': "B-" + annotation[1], 'subtype': annotation[2],
                      'set': sample['type'], 'start': annotation[3], 'index': data.index_finder(annotation[1])}
            annotated_sentence.append(_input)
            _input = {"word": _list[1], 'w-type': annotation[1], 'type': "L-" + annotation[1], 'subtype': annotation[2],
                      'set': sample['type'], 'start': annotation[3], 'index': data.index_finder(annotation[1])}
            annotated_sentence.append(_input)
        else:
            _input = {"word": _list[0], 'w-type': annotation[1], 'type': "B-" + annotation[1], 'subtype': annotation[2],
                      'set': sample['type'], 'start': annotation[3], 'index': data.index_finder(annotation[1])}
            annotated_sentence.append(_input)
            _input = {"word": _list[-1], 'w-type': annotation[1], 'type': "L-" + annotation[1], 'subtype': annotation[2],
                      'set': sample['type'], 'start': annotation[3], 'index': data.index_finder(annotation[1])}
            annotated_sentence.append(_input)
            for item in range(1, len(_list) - 1):
                _input = {"word": _list[item], 'w-type': annotation[1], 'type': "I-" + annotation[1], 'subtype': annotation[2],
                          'set': sample['type'], 'start': annotation[3], 'index': data.index_finder(annotation[1])}
                annotated_sentence.append(_input)
        start = start + value_index + len(_list)
        example['after'] = start
        example['boundary'] = [example['before']+example['value_index'], example['after'] - 1]
        checked = iteration+1
        phrases.append(example)
    print(phrases)
    print(count)
    for _it in range(start, len(sentence)):
            _input = {"word": sentence[_it], 'w-type': '-O-', 'type': '-O-', 'subtype': '-O-',
                      'set': sample['type'], 'index': data.index_finder('-O-')}
            annotated_sentence.append(_input)
    # annotated_sentence is an array of annotated words inside a sentence
    relations = []
    all_relations = []
    phrases_ids = [phrase['id'] for phrase in phrases]
    for phrase in phrases:
        for rel in sample['relations']:
            if rel[0] == phrase['id'] and rel[1] in phrases_ids:
                relations.append(rel)
            elif rel[0] == phrase['id'] or rel[1] == phrase['id']:
                all_relations.append(rel)

    sample['inputs'].append(annotated_sentence)
    sample['phrases'].append(phrases)
    total_phrases += len(phrases)
print(total_phrases+passed)
print(len(sample['annotated']))
print(passed_items)
print(before_passed)

In [20]:
results = data.data

In [21]:
update_list = []
count = 0
passing = 0
removing = []
print(len(data.data))
for _iter in range(len(data.data)):
    if count < passing:
        count += 1
        continue
    else:
        count = 0
        passing = 0
    if len(data.data[_iter]['all_relations']) != 0:
        print("iteration is ", _iter)
        for relation in data.data[_iter]['all_relations']:
            if relation in data.data[_iter]['relations']:
                print("here it is ")
                continue
            search = None
            check = False
            if relation[0] in [phrase['id'] for phrase in data.data[_iter]['phrases']]:
                search = relation[1]
            else:
                search = relation[0]
            print(search)
            for j in range(1,3):
                if search in [phrase['id'] for phrase in data.data[_iter+j]['phrases']]:
                    print("it is in ", _iter+j)
                    check = True
                    new_start = len(data.data[_iter]['words'])
                    passing = j
                    for i in range(1, j+1):
                        print("merging in process")
                        removing.append(_iter+i)
                        data.data[_iter]['words'].extend(data.data[_iter+i]['words'])
                        for phrase in data.data[_iter+i]['phrases']:
                            phrase['before'] += new_start
                        data.data[_iter]['phrases'].extend(data.data[_iter+i]['phrases'])
                        data.data[_iter]['relations'].extend(data.data[_iter+i]['relations'])
                        data.data[_iter]['all_relations'].extend(data.data[_iter+i]['all_relations'])
                        data.data[_iter]['sentence'] = " ".join([data.data[_iter]['sentence'], data.data[_iter+i]['sentence']])
                        new_start += len(data.data[_iter+i]['words'])
                    break
            if not check:
                print(search)
#                 print(relation)
                print(data.data[_iter]['sentence'])
#                 print(data.data[_iter]['relations'])
#                 print(data.data[_iter]['all_relations'])
                print("error")
            break
                
# for remove in removing:
#     del(data.data[remove])
print(len(data.data))              
            
                        

15651
iteration is  71
CNN_IP_20030402.1600.02-2-E13-33
it is in  72
merging in process
iteration is  156
CNN_IP_20030412.1600.03-E2-148
it is in  157
merging in process
iteration is  771
CNN_LE_20030504.1200.02-1-E40-97
CNN_LE_20030504.1200.02-1-E40-97
so i would n t dwell on that point specifically because saddam hussein clearly was a producer and a user of biological and chemical weapons
error
iteration is  822
CNN_IP_20030405.1600.02-E11-19
it is in  823
merging in process
iteration is  914
CNN_IP_20030403.1600.00-2-E16-106
it is in  915
merging in process
iteration is  983
CNN_CF_20030303.1900.00-E70-123
it is in  984
merging in process
iteration is  1197
CNN_IP_20030329.1600.01-1-E2-99
CNN_IP_20030329.1600.01-1-E2-99
judy wedeman ben wedeman describing coalition hits in the area where he is the town of kalak in northern iraq
error
iteration is  1544
CNN_IP_20030328.1600.07-E35-68
it is in  1545
merging in process
iteration is  1614
CNN_IP_20030408.1600.04-E102-145
it is in  1615


In [22]:
for remove in removing:
    del(data.data[remove])

In [23]:
for item in data.data:
    phrase_ids = [phrase['id'] for phrase in item['phrases']]
    for relation in item['relations']:
        if relation[0] not in phrase_ids or relation[1] not in phrase_ids:
            print(item)

In [24]:
train = [item for item in data.data if item['words'][0]['set'] == "train"]
valid = [item for item in data.data if item['words'][0]['set'] == "valid"]
test = [item for item in data.data if item['words'][0]['set'] == "test"]
print(len(train))
print(len(valid))
print(len(test))
print(len(data.data))
print(len([item for item in data.data if item['words'][0]['set'] == "ignore"]))
items = [item["words"] for item in data.data]
import itertools
merged = list(itertools.chain.from_iterable(items))
ratios = [0, 0, 0, 0, 0, 0, 0, 0]
def index_finder(_class, choices=["ORG", "FAC", "PER", "VEH", "LOC", "WEA", "GPE", "-O-"]):
        return choices.index(_class)

print(len([item for item in merged if item['w-type'] != "-O-"]))
_sum = 0
for item in merged:
    ratios[index_finder(item['w-type'])] += 1
    _sum += 1
for _it in range(len(ratios)):
    ratios[_it] = 1- (ratios[_it] / _sum)
print(ratios)
print(_sum)

10361
2607
2638
15606
0
70005
[0.9633815356778724, 0.9895314272584976, 0.8705106480750175, 0.9934014963309735, 0.991685454571564, 0.9944282493502018, 0.9457400519838591, 0.25132113675201406]
278548


In [30]:
it = 0
for it in range(int(len(train) / 1000)):
    with open('ACE_JSON/train/result'+str(it)+'.json', 'w+') as fp:
        json.dump(train[it*1000:(it+1)*1000], fp)
if it:
    number = it+1
else:
    number = it
with open('ACE_JSON/train/result'+str(number)+'.json', 'w+') as fp:
        json.dump(train[(number)*1000:], fp)
        
it = 0
for it in range(int(len(valid) / 1000)):
    with open('ACE_JSON/valid/result'+str(it)+'.json', 'w+') as fp:
        json.dump(valid[it*1000:(it+1)*1000], fp)
if it:
    number = it+1
else:
    number = it
with open('ACE_JSON/valid/result'+str(number)+'.json', 'w+') as fp:
        json.dump(valid[(number)*1000:], fp)

it = 0
for it in range(int(len(test) / 1000)):
    with open('ACE_JSON/test/result'+str(it)+'.json', 'w+') as fp:
        json.dump(test[it*1000:(it+1)*1000], fp)
if it:
    number = it+1
else:
    number = it
with open('ACE_JSON/test/result'+str(number)+'.json', 'w+') as fp:
        json.dump(test[(number)*1000:], fp)